Import Libraries

In [3]:
import xgboost as xgb
from sklearn import metrics
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import KFold, train_test_split
import numpy as np
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import Model
import cv2
import os

Load and preprocess Data

In [4]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), 1)
        img = cv2.resize(img,(224,224))
        img = preprocess_input(img)
        if img is not None:
            images.append(img)
    return images

corona = load_images_from_folder("./COVID-19-master/X-Ray Image DataSet/Covid-19")
no_find = load_images_from_folder("./COVID-19-master/X-Ray Image DataSet/No_findings")
total = corona + no_find 

label0 = []
for j in range(len(corona)):
  label0.append(0)

label1 = []
for j in range(len(no_find)):
  label1.append(1)

label = label0 + label1

dataset = np.array(total)
label = np.array(label)

Feature Extracting

In [ ]:
modeldensenet = tf.keras.applications.DenseNet201()

In [7]:
modeldn = Model(inputs=modeldensenet.inputs, outputs=modeldensenet.layers[-2].output)
xtrain, xtest, ytrain, ytest = train_test_split(dataset, label, test_size=0.2, random_state= 47)

feature_extractor= modeldn.predict(dataset)
Xfull = feature_extractor.reshape(feature_extractor.shape[0], -1)

feature_extractor= modeldn.predict(xtrain)
xtrain = feature_extractor.reshape(feature_extractor.shape[0], -1)

Classification

In [ ]:
model = xgb.XGBClassifier(booster = 'gbtree',
                           objective = "binary:logitraw",
                           reg_lambda = 1,
                           gamma = 0,
                           min_child_weight = 2,
                           n_estimators = 200,
                           base_score = 0.8,
                           learning_rate=0.157,
                           max_depth=6,)

model.fit(xtrain, ytrain)

X_test_feature = modeldn.predict(xtest)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)
prediction = model.predict(X_test_features)

print("Accuracy = ", metrics.accuracy_score(ytest, prediction))

In [ ]:
from sklearn.metrics import *

cv = KFold(n_splits=5, shuffle=True, random_state=39)
avgacc = 0
f = 0
for train_ix, test_ix in cv.split(Xfull):
    result={}
    train_X, test_X = Xfull[train_ix], Xfull[test_ix]
    train_y, Y_test = label[train_ix], label[test_ix]
    model.fit(train_X, train_y)
    Y_pred = model.predict(test_X)
    print(f"\nConfusion matrix fold {f} :")
    f = f+1
    result["Accuracy score"] = accuracy_score(Y_test, Y_pred)
    avgacc += result["Accuracy score"]
    result["confusion matrix"] = confusion_matrix(Y_test, Y_pred)
    result["f1 score"] = f1_score(Y_test, Y_pred, average="macro")
    result["precision score"] = precision_score(Y_test, Y_pred, average="macro")
      
    result["Sensitivity"] = result["confusion matrix"][0,0] / (result["confusion matrix"][0,0] + result["confusion matrix"][0,1])
    result["Specifity"] = result["confusion matrix"][1,1] / (result["confusion matrix"][1,0] + result["confusion matrix"][1,1])
      
    print(f"\nAccuracy score : ", result["Accuracy score"])
    print(f"\nconfusion matrix : \n", result["confusion matrix"])
    print(f"\nf1 score : ", result["f1 score"])
    print(f"\nprecision score : ", result["precision score"])
      
    print(f"\nSpecifity : ", result["Specifity"])
    print(f"\nSensitivity : ", result["Sensitivity"])
    print(f"\n================================================================")
print(avgacc/5)    